Grating
==
We simulate the scattering on a grating

In [1]:
import netgen.gui
from ngsolve import *

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2405
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 8 thread(s)


2024-10-25 20:45:11.259 python[49700:7794042] +[IMKClient subclass]: chose IMKClient_Legacy


In [2]:
from netgen.geom2d import *

geo = SplineGeometry()
geo.AddRectangle ( (0,0), (1,1), bcs=["bottom","right","top","left"], leftdomain=1)

n = 40
r = 0.005
for i in range(n-1):
    geo.AddCircle ( (0.5,(i+1)/(n)), r=r, bc="interface", 
                     leftdomain=0, rightdomain=1)
geo.SetMaterial(1, "medium")
# geo.SetMaterial(2, "dot")

mesh = Mesh(geo.GenerateMesh(maxh=0.02))
mesh.Curve(3)
Draw (mesh)

 Generate Mesh from spline geometry
 Curve elements, order = 3


In [3]:
fes = H1(mesh, order=6, complex=True, dirichlet="interface")
u = fes.TrialFunction()
v = fes.TestFunction()
a = BilinearForm(fes)
f = LinearForm(fes)

from math import pi
omega = 2*pi*42

n = { "medium" : 1, "dot" : 3 }
cfn = CoefficientFunction( [n[mat] for mat in mesh.GetMaterials()] )

a += (grad(u)*grad(v)-omega**2*cfn*cfn*u*v)*dx
a += -1j*omega*u*v*ds("left|right|top|bottom")
f = LinearForm(fes)

source = exp(-10**2*(y-0.5)*(y-0.5))
f += source*v*ds("left")

gfu = GridFunction(fes)


with TaskManager():
    a.Assemble()
    f.Assemble()
    inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
    gfu.vec.data = inv * f.vec
    
Draw (gfu)

Thank you for using NGSolve
